
詞對的社會網絡分析
* 詞對分析找出會共同出現的字詞。
* 字詞會共同出現一定有道理，我們姑且稱之為潛在語意，有個潛在語意在後面，把這些字詞放在一起。
* 語料庫是特定性質的字詞的集合，可以讓詞對分析更聚焦，更有目的性。
* 社會網絡分析則是希望以圖示的方式，更突顯潛在語意。

In [1]:
# 語料庫必須是字詞，不是句子。

In [2]:
from collections import Counter
import pandas as pd
import numpy
import numpy as np
import matplotlib.pyplot as plt
import os
import io
import re

# 中文設定1: 楷體字
from matplotlib import rcParams
rcParams["font.sans-serif"] = ["kaiu"] 
rcParams["font.family"] ="DFKai-sb"

In [3]:
!gdown --id '1Z1fAwZCsfmG3eiSdlgk5mHqhXUNMF0nK' --output news.xlsx
!gdown --id '1dBWYhSLR469VkwhwWV-yjbXf3bewonk2' --output deletewords.txt
!gdown --id '1fcjlTgN_CXXxIPAysafhXz8f0tI-1jgZ' --output userdict.txt
!wget -N https://raw.githubusercontent.com/P4CSS/PSS/master/data/stopwords_zh-tw.txt
with open("stopwords_zh-tw.txt", encoding="utf-8") as fin:
    stopwords = fin.read().split("\n")[1:]

Downloading...
From: https://drive.google.com/uc?id=1Z1fAwZCsfmG3eiSdlgk5mHqhXUNMF0nK
To: /content/news.xlsx
100% 13.1M/13.1M [00:00<00:00, 58.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1dBWYhSLR469VkwhwWV-yjbXf3bewonk2
To: /content/deletewords.txt
100% 8.12k/8.12k [00:00<00:00, 13.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1fcjlTgN_CXXxIPAysafhXz8f0tI-1jgZ
To: /content/userdict.txt
100% 31.0/31.0 [00:00<00:00, 44.2kB/s]
--2022-03-06 17:07:58--  https://raw.githubusercontent.com/P4CSS/PSS/master/data/stopwords_zh-tw.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8125 (7.9K) [text/plain]
Saving to: ‘stopwords_zh-tw.txt’

stopwords_zh-tw.txt 100%[===================>]   7.93K  --.-KB/s    in 0s      

Last-modified header m

# 讀取資料

In [11]:
df = pd.read_excel('news.xlsx')
df.columns = ['index', +'dates', 'title', 'address', 'content',]
df

,index,dates,title,address,content
0,0,2020-12-29,比特幣漲破兩萬美金之後呢？方舟投資女股神Cathie Wood：極度看漲,https://zombit.info/after-bitcoin-surpassed-20...,專注於顛覆式新創的投資機構「方舟投資 (Ark Invest) 」看好比特幣已不是新聞，不過...
1,1,2021-01-01,瑞波官司餘震！eToro 即將關閉 XRP 交易、Coinbase 遭控販售 XRP 面臨「...,https://www.blocktempo.com/ripple-lawsuit-coin...,\n瑞波公司（Ripple）遭美國證券交易委員會起訴案，鬧得滿城風雨。已經有十幾個交易平台下...
2,2,2021-01-01,機構進場新血｜那斯達克上市「綠專資本」計畫發 1 億美元債券，收益將投資比特幣,https://www.blocktempo.com/greenpro-capital-ra...,\n機構進場比特幣即將再多一名新血！美國那斯達克上市公司綠專資本（Greenpro Capi...
3,3,2021-01-01,USDT是下個瑞波？Messari : Tether是市場的最大威脅，「生死交關」的 202...,https://www.blocktempo.com/tether-and-stableco...,\n錨定美元的穩定幣 USDT 因價格波動小，為加密市場重要交易媒介，也是市值僅次於比特幣（...
4,4,2021-01-02,"加密億萬富翁 Mike Novogratz 大哉問：臉書DIEM, BCH, USDT, U...",https://www.blocktempo.com/9-trillion-vs-100-t...,\n拜各國央行打開印鈔機開關所賜，在注意到如債券等傳統配置並不能對沖通膨風險後，機構、投資人...
...,...,...,...,...,...
5989,5989,2021-12-20,Amber Group 榮獲「2021德勤香港高科技高成長領軍企業」獎項,https://zombit.info/amber-group-named-a-winner...,12 月 15 日，2021 年「德勤香港高科技高成長」頒獎典禮在香港隆重舉行，數字金融科技...
5990,5990,2021-12-21,備受矚目的 3A 級遊戲大作 Big Time 將開啟 SPACE NFT Sale,https://www.grenade.tw/blog/big-time-space-nft...,"\n\n\n\n發佈於 21 12 月, 202121 12 月, 2021 作者: Gre..."
5991,5991,2021-12-21,美國政府從執法行動中沒收了大量的比特幣庫存，背後的動向如何運作？,https://zombit.info/government-stockpile-bitco...,美國政府從犯罪活動中扣押加密貨幣之後，會定期舉行法拍來出售被其扣押的比特幣（BTC）、以太坊...
5992,5992,2021-12-21,Footprint Analytics:去中心化存儲協定是否能取代中心化雲存儲？,https://www.grenade.tw/blog/footprint-analytic...,"\n\n\n\n發佈於 21 12 月, 202121 12 月, 2021 作者: Foo..."


In [12]:
df.head(10)

,index,dates,title,address,content
0,0,2020-12-29,比特幣漲破兩萬美金之後呢？方舟投資女股神Cathie Wood：極度看漲,https://zombit.info/after-bitcoin-surpassed-20...,專注於顛覆式新創的投資機構「方舟投資 (Ark Invest) 」看好比特幣已不是新聞，不過...
1,1,2021-01-01,瑞波官司餘震！eToro 即將關閉 XRP 交易、Coinbase 遭控販售 XRP 面臨「...,https://www.blocktempo.com/ripple-lawsuit-coin...,\n瑞波公司（Ripple）遭美國證券交易委員會起訴案，鬧得滿城風雨。已經有十幾個交易平台下...
2,2,2021-01-01,機構進場新血｜那斯達克上市「綠專資本」計畫發 1 億美元債券，收益將投資比特幣,https://www.blocktempo.com/greenpro-capital-ra...,\n機構進場比特幣即將再多一名新血！美國那斯達克上市公司綠專資本（Greenpro Capi...
3,3,2021-01-01,USDT是下個瑞波？Messari : Tether是市場的最大威脅，「生死交關」的 202...,https://www.blocktempo.com/tether-and-stableco...,\n錨定美元的穩定幣 USDT 因價格波動小，為加密市場重要交易媒介，也是市值僅次於比特幣（...
4,4,2021-01-02,"加密億萬富翁 Mike Novogratz 大哉問：臉書DIEM, BCH, USDT, U...",https://www.blocktempo.com/9-trillion-vs-100-t...,\n拜各國央行打開印鈔機開關所賜，在注意到如債券等傳統配置並不能對沖通膨風險後，機構、投資人...
5,5,2021-01-02,比特幣漲勢兇猛 一舉突破30000美元大關續寫新高,https://news.cnyes.com/news/id/4555735?exp=a,變種病毒讓全球疫情再次升溫，加上各國為刺激經濟持續推出貨幣寬鬆政策，美元走弱也促使熱錢湧入虛...
6,6,2021-01-03,隱私幣｜達世幣 DASH 宣布測試 v 0.17 版本、重點產品「DashPay」推出在即？,https://www.blocktempo.com/dash-announces-new-...,\n隨著加密貨幣走入主流大眾視野，隱私幣監管似乎也成為一大問題，不過這不影響隱私幣的開發節奏...
7,7,2021-01-03,"漲破三萬！比特幣新高來到33,300，市值破6千億鎂超車「台積電」登第九大資產",https://www.blocktempo.com/bitcoin-hits-30k-fo...,"\n在社群殷殷期盼下，比特幣（BTC）於昨日正式突破 30,000 美元的大關。在突破 30..."
8,8,2021-01-03,不滿SEC無視FinCEN 認「XRP為貨幣」！瑞波支持者發起白宮請願，破10萬人白宮就得回應,https://www.blocktempo.com/xrp-enthusiasts-see...,\n從瑞波公司和其兩位共同創辦人遭到美國證券交易委員會（SEC）起訴後，瑞波幣（XRP）價格...
9,9,2021-01-03,馬斯克最愛｜狗狗幣衝漲173%、創兩年來新高！DOGE躋身市值第22大加密貨幣,https://www.blocktempo.com/dogecoin-to-the-mar...,"\n科技狂人馬斯克（Elon Musk）最愛的狗狗幣（DOGE）又來了！在比特幣直衝 33,..."


# 讀取停用字

In [13]:
file_name = "deletewords.txt"
deletewords = open(file_name,'r',encoding='utf-8').read()
deletewords=list(set(deletewords.split()))

In [14]:
df

,index,dates,title,address,content
0,0,2020-12-29,比特幣漲破兩萬美金之後呢？方舟投資女股神Cathie Wood：極度看漲,https://zombit.info/after-bitcoin-surpassed-20...,專注於顛覆式新創的投資機構「方舟投資 (Ark Invest) 」看好比特幣已不是新聞，不過...
1,1,2021-01-01,瑞波官司餘震！eToro 即將關閉 XRP 交易、Coinbase 遭控販售 XRP 面臨「...,https://www.blocktempo.com/ripple-lawsuit-coin...,\n瑞波公司（Ripple）遭美國證券交易委員會起訴案，鬧得滿城風雨。已經有十幾個交易平台下...
2,2,2021-01-01,機構進場新血｜那斯達克上市「綠專資本」計畫發 1 億美元債券，收益將投資比特幣,https://www.blocktempo.com/greenpro-capital-ra...,\n機構進場比特幣即將再多一名新血！美國那斯達克上市公司綠專資本（Greenpro Capi...
3,3,2021-01-01,USDT是下個瑞波？Messari : Tether是市場的最大威脅，「生死交關」的 202...,https://www.blocktempo.com/tether-and-stableco...,\n錨定美元的穩定幣 USDT 因價格波動小，為加密市場重要交易媒介，也是市值僅次於比特幣（...
4,4,2021-01-02,"加密億萬富翁 Mike Novogratz 大哉問：臉書DIEM, BCH, USDT, U...",https://www.blocktempo.com/9-trillion-vs-100-t...,\n拜各國央行打開印鈔機開關所賜，在注意到如債券等傳統配置並不能對沖通膨風險後，機構、投資人...
...,...,...,...,...,...
5989,5989,2021-12-20,Amber Group 榮獲「2021德勤香港高科技高成長領軍企業」獎項,https://zombit.info/amber-group-named-a-winner...,12 月 15 日，2021 年「德勤香港高科技高成長」頒獎典禮在香港隆重舉行，數字金融科技...
5990,5990,2021-12-21,備受矚目的 3A 級遊戲大作 Big Time 將開啟 SPACE NFT Sale,https://www.grenade.tw/blog/big-time-space-nft...,"\n\n\n\n發佈於 21 12 月, 202121 12 月, 2021 作者: Gre..."
5991,5991,2021-12-21,美國政府從執法行動中沒收了大量的比特幣庫存，背後的動向如何運作？,https://zombit.info/government-stockpile-bitco...,美國政府從犯罪活動中扣押加密貨幣之後，會定期舉行法拍來出售被其扣押的比特幣（BTC）、以太坊...
5992,5992,2021-12-21,Footprint Analytics:去中心化存儲協定是否能取代中心化雲存儲？,https://www.grenade.tw/blog/footprint-analytic...,"\n\n\n\n發佈於 21 12 月, 202121 12 月, 2021 作者: Foo..."


# 資料清理
把不是文字的內文用空值取代

In [15]:
content = []
for i,j in zip(df['content'], range(len(df['content']))):
  # print(i)
  if type(i)!=str or i=='':
    content.append('')
  else:
    content.append(i.replace('\n',''))
df['content'] = content

## 斷詞

In [16]:
import jieba
jieba.load_userdict('userdict.txt') #匯入自訂字典

df['content_token'] = df['content'].apply(lambda x:list(jieba.cut(x)))
df

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.853 seconds.
Prefix dict has been built successfully.


,index,dates,title,address,content,content_token
0,0,2020-12-29,比特幣漲破兩萬美金之後呢？方舟投資女股神Cathie Wood：極度看漲,https://zombit.info/after-bitcoin-surpassed-20...,專注於顛覆式新創的投資機構「方舟投資 (Ark Invest) 」看好比特幣已不是新聞，不過...,"[專注, 於, 顛覆式, 新創, 的, 投資, 機構, 「, 方舟, 投資, , (, A..."
1,1,2021-01-01,瑞波官司餘震！eToro 即將關閉 XRP 交易、Coinbase 遭控販售 XRP 面臨「...,https://www.blocktempo.com/ripple-lawsuit-coin...,瑞波公司（Ripple）遭美國證券交易委員會起訴案，鬧得滿城風雨。已經有十幾個交易平台下架或...,"[瑞波, 公司, （, Ripple, ）, 遭美國, 證券, 交易, 委員會, 起, 訴案..."
2,2,2021-01-01,機構進場新血｜那斯達克上市「綠專資本」計畫發 1 億美元債券，收益將投資比特幣,https://www.blocktempo.com/greenpro-capital-ra...,機構進場比特幣即將再多一名新血！美國那斯達克上市公司綠專資本（Greenpro Capita...,"[機構, 進場, 比特幣, 即將, 再, 多, 一名, 新血, ！, 美國, 那斯達克, 上..."
3,3,2021-01-01,USDT是下個瑞波？Messari : Tether是市場的最大威脅，「生死交關」的 202...,https://www.blocktempo.com/tether-and-stableco...,錨定美元的穩定幣 USDT 因價格波動小，為加密市場重要交易媒介，也是市值僅次於比特幣（BT...,"[錨定, 美元, 的, 穩定幣, , USDT, , 因價格, 波動, 小, ，, 為,..."
4,4,2021-01-02,"加密億萬富翁 Mike Novogratz 大哉問：臉書DIEM, BCH, USDT, U...",https://www.blocktempo.com/9-trillion-vs-100-t...,拜各國央行打開印鈔機開關所賜，在注意到如債券等傳統配置並不能對沖通膨風險後，機構、投資人開始...,"[拜, 各國, 央行, 打開, 印鈔, 機開關, 所, 賜, ，, 在, 注意, 到, 如,..."
...,...,...,...,...,...,...
5989,5989,2021-12-20,Amber Group 榮獲「2021德勤香港高科技高成長領軍企業」獎項,https://zombit.info/amber-group-named-a-winner...,12 月 15 日，2021 年「德勤香港高科技高成長」頒獎典禮在香港隆重舉行，數字金融科技...,"[12, , 月, , 15, , 日, ，, 2021, , 年, 「, 德勤, ..."
5990,5990,2021-12-21,備受矚目的 3A 級遊戲大作 Big Time 將開啟 SPACE NFT Sale,https://www.grenade.tw/blog/big-time-space-nft...,"發佈於 21 12 月, 202121 12 月, 2021 作者: Grenade, 閱讀...","[發, 佈, 於, , 21, , 12, , 月, ,, , 202121, ,..."
5991,5991,2021-12-21,美國政府從執法行動中沒收了大量的比特幣庫存，背後的動向如何運作？,https://zombit.info/government-stockpile-bitco...,美國政府從犯罪活動中扣押加密貨幣之後，會定期舉行法拍來出售被其扣押的比特幣（BTC）、以太坊...,"[美國, 政府, 從, 犯罪, 活動, 中, 扣押, 加密, 貨幣, 之, 後, ，, 會,..."
5992,5992,2021-12-21,Footprint Analytics:去中心化存儲協定是否能取代中心化雲存儲？,https://www.grenade.tw/blog/footprint-analytic...,"發佈於 21 12 月, 202121 12 月, 2021 作者: Footprint, ...","[發, 佈, 於, , 21, , 12, , 月, ,, , 202121, ,..."


## 清除標點符號


In [17]:
import unicodedata # for removing Chinese puctuation
def remove_punc_by_unicode(words):
    out = []
    for word in words:
        if word != " " and not unicodedata.category(word[0]).startswith('P'):
            out.append(word)
    return out


df['cleaned_content_token'] = df['content_token'].apply(remove_punc_by_unicode)

In [18]:
df

,index,dates,title,address,content,content_token,cleaned_content_token
0,0,2020-12-29,比特幣漲破兩萬美金之後呢？方舟投資女股神Cathie Wood：極度看漲,https://zombit.info/after-bitcoin-surpassed-20...,專注於顛覆式新創的投資機構「方舟投資 (Ark Invest) 」看好比特幣已不是新聞，不過...,"[專注, 於, 顛覆式, 新創, 的, 投資, 機構, 「, 方舟, 投資, , (, A...","[專注, 於, 顛覆式, 新創, 的, 投資, 機構, 方舟, 投資, Ark, Inves..."
1,1,2021-01-01,瑞波官司餘震！eToro 即將關閉 XRP 交易、Coinbase 遭控販售 XRP 面臨「...,https://www.blocktempo.com/ripple-lawsuit-coin...,瑞波公司（Ripple）遭美國證券交易委員會起訴案，鬧得滿城風雨。已經有十幾個交易平台下架或...,"[瑞波, 公司, （, Ripple, ）, 遭美國, 證券, 交易, 委員會, 起, 訴案...","[瑞波, 公司, Ripple, 遭美國, 證券, 交易, 委員會, 起, 訴案, 鬧得, ..."
2,2,2021-01-01,機構進場新血｜那斯達克上市「綠專資本」計畫發 1 億美元債券，收益將投資比特幣,https://www.blocktempo.com/greenpro-capital-ra...,機構進場比特幣即將再多一名新血！美國那斯達克上市公司綠專資本（Greenpro Capita...,"[機構, 進場, 比特幣, 即將, 再, 多, 一名, 新血, ！, 美國, 那斯達克, 上...","[機構, 進場, 比特幣, 即將, 再, 多, 一名, 新血, 美國, 那斯達克, 上市公司..."
3,3,2021-01-01,USDT是下個瑞波？Messari : Tether是市場的最大威脅，「生死交關」的 202...,https://www.blocktempo.com/tether-and-stableco...,錨定美元的穩定幣 USDT 因價格波動小，為加密市場重要交易媒介，也是市值僅次於比特幣（BT...,"[錨定, 美元, 的, 穩定幣, , USDT, , 因價格, 波動, 小, ，, 為,...","[錨定, 美元, 的, 穩定幣, USDT, 因價格, 波動, 小, 為, 加密, 市場, ..."
4,4,2021-01-02,"加密億萬富翁 Mike Novogratz 大哉問：臉書DIEM, BCH, USDT, U...",https://www.blocktempo.com/9-trillion-vs-100-t...,拜各國央行打開印鈔機開關所賜，在注意到如債券等傳統配置並不能對沖通膨風險後，機構、投資人開始...,"[拜, 各國, 央行, 打開, 印鈔, 機開關, 所, 賜, ，, 在, 注意, 到, 如,...","[拜, 各國, 央行, 打開, 印鈔, 機開關, 所, 賜, 在, 注意, 到, 如, 債券..."
...,...,...,...,...,...,...,...
5989,5989,2021-12-20,Amber Group 榮獲「2021德勤香港高科技高成長領軍企業」獎項,https://zombit.info/amber-group-named-a-winner...,12 月 15 日，2021 年「德勤香港高科技高成長」頒獎典禮在香港隆重舉行，數字金融科技...,"[12, , 月, , 15, , 日, ，, 2021, , 年, 「, 德勤, ...","[12, 月, 15, 日, 2021, 年, 德勤, 香港, 高科技, 高成, 長, 頒獎..."
5990,5990,2021-12-21,備受矚目的 3A 級遊戲大作 Big Time 將開啟 SPACE NFT Sale,https://www.grenade.tw/blog/big-time-space-nft...,"發佈於 21 12 月, 202121 12 月, 2021 作者: Grenade, 閱讀...","[發, 佈, 於, , 21, , 12, , 月, ,, , 202121, ,...","[發, 佈, 於, 21, 12, 月, 202121, 12, 月, 2021, 作者, ..."
5991,5991,2021-12-21,美國政府從執法行動中沒收了大量的比特幣庫存，背後的動向如何運作？,https://zombit.info/government-stockpile-bitco...,美國政府從犯罪活動中扣押加密貨幣之後，會定期舉行法拍來出售被其扣押的比特幣（BTC）、以太坊...,"[美國, 政府, 從, 犯罪, 活動, 中, 扣押, 加密, 貨幣, 之, 後, ，, 會,...","[美國, 政府, 從, 犯罪, 活動, 中, 扣押, 加密, 貨幣, 之, 後, 會, 定期..."
5992,5992,2021-12-21,Footprint Analytics:去中心化存儲協定是否能取代中心化雲存儲？,https://www.grenade.tw/blog/footprint-analytic...,"發佈於 21 12 月, 202121 12 月, 2021 作者: Footprint, ...","[發, 佈, 於, , 21, , 12, , 月, ,, , 202121, ,...","[發, 佈, 於, 21, 12, 月, 202121, 12, 月, 2021, 作者, ..."


# 字頻分析

In [22]:
restore = []
for i in df['cleaned_content_token']:
  for j in i:
    restore.append(j)


In [23]:
from collections import Counter
tf = Counter()
for sentence in restore:
  tf[sentence] += 1

filtered = dict(tf)

In [24]:
def removeAlphaNum(i): #去除英文與數字
  temp_ = []
  for sentence in i:
    if '\u4e00' <= sentence <= '\u9fa5':
      continue
    else:
      temp_.append(sentence)  
  for i in temp_:
    del filtered[i]
  
  return filtered



def removeChiStopWords(i):
  temp_ = []
  for sentence in i:
    if sentence in stopwords:
      temp_.append(sentence)
    else:
      continue  
  for i in temp_:
    del filtered[i]
  
  return filtered

filtered = removeChiStopWords(filtered)
filtered = removeAlphaNum(filtered)
print(filtered)




{'專注': 757, '顛覆式': 3, '新創': 364, '投資': 10171, '機構': 5717, '方舟': 424, '看好': 680, '比特幣': 28627, '已': 6288, '新聞': 545, '執行長': 1934, '近日': 592, '接受': 2204, '彭博': 207, '專訪': 292, '時': 6198, '解釋': 690, '原因': 1460, '並對': 143, '看漲': 500, '成立': 1306, '年': 11979, '特斯拉': 1725, '樂觀': 319, '前景': 351, '而聞名': 39, '二十大': 7, '股東': 181, '其創': 11, '辦人': 55, '更因': 1, '獨到': 17, '眼光': 44, '操盤': 29, '績效': 48, '卓越': 80, '譽': 45, '女版': 18, '巴菲特': 153, '熊市': 263, '證明': 1549, '更將': 25, '稱為': 77, '加密': 26921, '生態': 5749, '系統': 3997, '中': 17125, '儲備': 861, '貨幣': 26283, '問道': 25, '再次': 731, '突破': 1542, '萬大關': 13, '看法': 540, '當時': 1328, '價位': 165, '約': 4921, '美金': 761, '表明': 670, '指出': 2893, '以來': 2306, '信心': 401, '當我們': 117, '看到': 2099, '跌至': 171, '多美元': 21, '隨後': 288, '主導': 402, '地位': 820, '左右': 849, '增長': 3273, '意味著': 1017, '心裡': 9, '確定': 711, '系中': 25, '強調': 708, '過程': 1599, '成為': 2753, '最': 4384, '安全': 2832, '這是': 2422, '一個': 15215, '重要': 2908, '角色': 1033, '背': 593, '後': 13767, '區塊': 15455, '鏈技術': 1900, '所有': 5

In [25]:
Counter(filtered).most_common(200)
# [('美元', 29427),
#  ('比特幣', 28627),
#  ('加密', 26921),
#  ('貨幣', 26283),
#  ('交易', 20393),
#  ('市場', 17092),
#  ('區塊', 15455)]

[('美元', 29427),
 ('比特幣', 28627),
 ('加密', 26921),
 ('貨幣', 26283),
 ('交易', 20393),
 ('月', 20176),
 ('爲', 17416),
 ('中', 17125),
 ('市場', 17092),
 ('區塊', 15455),
 ('一個', 15215),
 ('鏈', 14553),
 ('以太', 14129),
 ('後', 13767),
 ('用戶', 13356),
 ('年', 11979),
 ('資產', 11471),
 ('人', 11123),
 ('坊', 10907),
 ('億', 10226),
 ('公司', 10201),
 ('投資', 10171),
 ('日', 9918),
 ('萬', 9899),
 ('提供', 9868),
 ('代幣', 9616),
 ('進行', 9265),
 ('表示', 9164),
 ('目前', 9092),
 ('項目', 8317),
 ('價格', 8129),
 ('數據', 8128),
 ('協議', 8098),
 ('遊戲', 7607),
 ('說', 7548),
 ('使用', 7465),
 ('中心化', 7460),
 ('交易所', 7350),
 ('金融', 7266),
 ('平台', 6631),
 ('時間', 6461),
 ('需要', 6317),
 ('已', 6288),
 ('時', 6198),
 ('基金', 6172),
 ('價值', 6147),
 ('下', 6124),
 ('支付', 5979),
 ('獲得', 5949),
 ('流動性', 5874),
 ('美國', 5865),
 ('前', 5792),
 ('生態', 5749),
 ('新', 5745),
 ('機構', 5717),
 ('推出', 5676),
 ('發展', 5672),
 ('已經', 5310),
 ('數位', 5305),
 ('合作', 5276),
 ('所有', 5219),
 ('監管', 5089),
 ('未來', 5054),
 ('錢', 4995),
 ('世界', 4980),
 ('虛擬', 4979),
 (

## 只用前後一個字來算共現詞

In [26]:
def removeAlphaNum(i): #去除英文與數字
  temp_ = []
  for sentence in i:
    if '\u4e00' <= sentence <= '\u9fa5':
      
      temp_.append(sentence)
    else:
      continue 
  return temp_



def removeChiStopWords(i):
  temp_ = []
  for sentence in i:
    if sentence in stopwords:
      continue  
    else:
      temp_.append(sentence)  
  return temp_

restore = removeAlphaNum(restore)
restore = removeChiStopWords(restore)


In [27]:
word_pair_counts = Counter()
for i in range(len(restore) - 1):
    (w1, w2) = (restore[i], restore[i + 1])
    word_pair_counts[(w1, w2)] += 1
    
for pair, c in word_pair_counts.most_common(500):
    print("%s\t%s\t%d" % (pair[0], pair[1], c))

加密	貨幣	18773
以太	坊	10896
億	美元	9087
萬	美元	8014
區塊	鏈	7695
月	日	5920
延伸	閱讀	3344
年	月	3296
元	宇宙	3257
平	臺	2733
虛擬	貨幣	2623
前	情	2385
情	提要	2385
智能	合約	2043
塊	鏈	1967
創辦	人	1863
今年	月	1824
錢	包	1735
註	冊	1663
發	佈	1659
區塊	鏈技術	1653
解決	方案	1598
一篇	文章	1578
生態	系統	1547
貨幣	市場	1546
創始	人	1522
聯	繫	1509
貨幣	交易所	1480
下	一篇	1407
作	爲	1398
馬	斯克	1384
以太	幣	1348
基礎	設施	1341
情況	下	1231
進	一步	1208
事件	背景	1189
數位	貨幣	1185
越來	越	1152
數位	資產	1144
區塊	客	1131
比特幣	價格	1121
創	作者	1115
完成	萬	1112
鏈	區塊	1111
加密	資產	1097
拍	賣	1076
市場	動態	1057
超過	億	1052
內容	區塊	1052
合作	建議	1048
鏈技術	內容	1047
站	合作	1047
歡迎	請	1047
動態	新聞傳	1046
新聞傳	送門	1046
送門	區塊	1046
客	致力	1046
致力	發掘	1046
發掘	整理	1046
整理	區塊	1046
區塊	客網	1046
客網	站	1046
建議	歡迎	1046
請	發電郵至	1046
發電郵至	聯	1046
成	爲	1026
熱門	市場	1025
數據	顯示	1021
月	月	1011
投資	建議	1001
不構	成	969
成	投資	965
貨幣	區塊	934
中心化	交易所	923
宣	佈	919
監管	機構	888
長	期	875
黃	金	872
貨幣	交易	869
項	目的	857
追	蹤	842
分析	師	810
標	籤	790
佈	月	779
貨幣	波動	779
比特幣	以太	773
迴	響	770
最	後	765
去年	月	764
本文	不構	760
治理	代幣	758
掌握	虛擬	757
閱讀	時間	756
謹慎	小心	756
文章	一篇	756
月	作者	755
建議	虛擬	755
波動	大請	755
大請	謹慎	755

# 自訂window size算共現詞

In [28]:
# window size !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
w=3

In [29]:
# 計算配對的次數
num_bigrams = 0
word_pair_count=Counter() 
word_counts = Counter(restore)
for i in range(len(restore)-1):
    # w=3
    for j in range(1,w+1):     
        if i+j < len(restore):            
            (w1,w2)=restore[i],restore[i+j]   
            word_pair_count[(w1,w2)]+=1
            num_bigrams += 1          
print ("total bigrams", num_bigrams)

total bigrams 9241329


In [30]:
word_pair_count

Counter({('專注', '顛覆式'): 1,
         ('專注', '新創'): 2,
         ('專注', '投資'): 48,
         ('顛覆式', '新創'): 1,
         ('顛覆式', '投資'): 1,
         ('顛覆式', '機構'): 1,
         ('新創', '投資'): 8,
         ('新創', '機構'): 4,
         ('新創', '方舟'): 1,
         ('投資', '機構'): 417,
         ('投資', '方舟'): 23,
         ('投資', '投資'): 341,
         ('機構', '方舟'): 1,
         ('機構', '投資'): 185,
         ('機構', '看好'): 13,
         ('方舟', '投資'): 274,
         ('方舟', '看好'): 2,
         ('方舟', '比特幣'): 69,
         ('投資', '看好'): 16,
         ('投資', '比特幣'): 640,
         ('投資', '已'): 58,
         ('看好', '比特幣'): 121,
         ('看好', '已'): 4,
         ('看好', '新聞'): 1,
         ('比特幣', '已'): 288,
         ('比特幣', '新聞'): 15,
         ('比特幣', '執行長'): 74,
         ('已', '新聞'): 2,
         ('已', '執行長'): 6,
         ('已', '近日'): 3,
         ('新聞', '執行長'): 3,
         ('新聞', '近日'): 2,
         ('新聞', '接受'): 2,
         ('執行長', '近日'): 13,
         ('執行長', '接受'): 65,
         ('執行長', '彭博'): 3,
         ('近日', '接受'): 35,
   

## 計算卡方值

In [31]:
# 計算卡方值
def chisquare(o11, o12, o21, o22):
    n = o11 + o12 + o21 + o22
    x_2 = (n * ((o11 * o22 - o12 * o21)**2)) / ((o11 + o12) * (o11 + o21) * (o12 + o22) * (o21 + o22)) 
    return x_2
pair_chi_squares = Counter()
for (w1, w2), w1_w2_count in word_pair_count.most_common():
        # O12
        w1_only_count = word_counts[w1] - w1_w2_count
        # O21
        w2_only_count = word_counts[w2] - w1_w2_count
        # O22
        rest_count = num_bigrams - w1_only_count - w2_only_count - w1_w2_count
        # O11= w1_w2_count
        pair_chi_squares[(w1, w2)] = chisquare(w1_w2_count, w1_only_count, w2_only_count, rest_count)

In [32]:
Counter(pair_chi_squares).most_common(200)


[(('圖代號', '圖代號'), 29941920.36000952),
 (('莫納', '什大學'), 20792993.250000756),
 (('總面', '厘米'), 18482662.000001732),
 (('積為', '厘米'), 18482662.000001732),
 (('待付', '工程款'), 18482662.000001732),
 (('他統', '大篆'), 18482662.000001732),
 (('臥室', '掛一張'), 18482662.000001732),
 (('開始試', '負票'), 18482660.000000432),
 (('總不會', '連結會員卡'), 18482660.000000432),
 (('齒', '輪組'), 18482660.000000432),
 (('中開設', '首爾新創'), 18482660.000000432),
 (('改弱', '聖騎'), 18482660.000000432),
 (('太昂貴', '支應且'), 18482660.000000432),
 (('法系', '成文法'), 18482660.000000432),
 (('这样', '波动'), 18482660.000000432),
 (('滑鐵', '盧大學'), 14439579.06250119),
 (('監管方', '掩耳'), 13861998.000004059),
 (('厘米', '英寸'), 13861996.500001948),
 (('英寸', '中標者'), 12321774.666668542),
 (('海狸', '浪濤'), 12321774.666668542),
 (('操縱水', '粒子'), 12321774.666668542),
 (('西蒙斯', '再托舉'), 12321773.333333801),
 (('費之', '嚶'), 12321773.333333801),
 (('煞', '滿建議'), 12321773.333333801),
 (('射手', '攻擊輸'), 12321773.333333801),
 (('美國勞', '工部'), 12070309.551022248),
 (('隨手', '扔'), 108

## edges, 篩選合於條件者排序

## 設定條件，結合語料庫



In [33]:
kw=["比特幣", '狗狗幣']    # 可設多個比較 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! kw=["比特幣", '狗狗幣', ........] 

In [34]:
corpus = restore

In [35]:
W1=[]
W2=[]
weight=[]
for (w1, w2), x_2 in pair_chi_squares.most_common():
    # 設定條件 ##############################################    
    if (len(w1)>1 and len(w2)>1) and (w1 in kw or w2 in kw):   
#      if (w1 in kw and w2 in corpus) or  (w2 in kw and w1 in corpus):   
     ########################################################    
        W1.append(w1)
        W2.append(w2)
        weight.append(x_2)
# edges 變數很重要，等一下社會網絡圖會用到    
edges=list(zip(W1, W2, weight))
edges

[('比特幣', '期貨', 109995.43580936006),
 ('比特幣', '價格', 82999.32856807791),
 ('狗狗幣', '殺手', 54148.41730924654),
 ('狗狗幣', '教父', 47857.626521031314),
 ('持有', '比特幣', 44810.82355250318),
 ('狗狗幣', '起死回生', 29960.26789082438),
 ('接受', '比特幣', 28603.233884132827),
 ('表示', '比特幣', 28372.142364901098),
 ('斯克', '狗狗幣', 27152.59228622111),
 ('美元', '比特幣', 26175.70171732079),
 ('斯克上', '狗狗幣', 23763.600513341768),
 ('比特幣', '以太', 22964.70668530382),
 ('狗狗幣', '暴漲', 22466.46472947296),
 ('比特幣', '法定', 21963.58655360937),
 ('購入', '比特幣', 21884.8451638069),
 ('認為', '比特幣', 21650.97185485898),
 ('比特幣', '挖礦', 21176.451476649065),
 ('狗狗幣', '養柴', 20800.125115327228),
 ('狗狗幣', '反讓', 20800.125115327228),
 ('節禮物', '狗狗幣', 20800.125115327228),
 ('狗狗幣', '柴犬幣', 20732.70211877715),
 ('比特幣', '美元', 20263.918636117916),
 ('買進', '比特幣', 19930.79214105965),
 ('狗狗幣', '狗狗幣', 19551.416855103358),
 ('特斯拉', '比特幣', 19546.934514501314),
 ('推特才', '狗狗幣', 17828.676741046034),
 ('狗狗幣', '王圖', 17828.676741046034),
 ('狗狗幣', '月球', 17295.29078496986),

In [36]:
# 配對結果後複雜，可以再用語料庫減化

In [37]:
df=pd.DataFrame(edges,columns=["name1","name2","weight"])
df

,name1,name2,weight
0,比特幣,期貨,109995.435809
1,比特幣,價格,82999.328568
2,狗狗幣,殺手,54148.417309
3,狗狗幣,教父,47857.626521
4,持有,比特幣,44810.823553
...,...,...,...
26767,機制,狗狗幣,0.000052
26768,比特幣,運營,0.000034
26769,底層,比特幣,0.000027
26770,玩家,狗狗幣,0.000021


# 詞對與社會網絡分析 
* 社會網絡分析
* 每個字詞為一個節點
* 與其有關鍵的字詞有連結
* 連結愈強，線條愈粗。
* networkX  如果無法出現中文字型，可參考以下做法： 
 * 目錄 C:\Users\\[USERNAME]\anaconda3\Lib\site-packages\matplotlib\mpl-data\fonts\ttf
 * 舊檔 DejaVuSans.ttf 改名為 DejaVuSans_bak.ttf
 * 拷貝新檔  DejaVuSans.ttf （請從教學網頁下載）

# 第一層最重要的關係取十個，不夠再加

In [38]:
df1=df.iloc[:20]
df1

,name1,name2,weight
0,比特幣,期貨,109995.435809
1,比特幣,價格,82999.328568
2,狗狗幣,殺手,54148.417309
3,狗狗幣,教父,47857.626521
4,持有,比特幣,44810.823553
5,狗狗幣,起死回生,29960.267891
6,接受,比特幣,28603.233884
7,表示,比特幣,28372.142365
8,斯克,狗狗幣,27152.592286
9,美元,比特幣,26175.701717


In [39]:
# 最後修正，刪除不要

In [40]:
df1.drop([18, 13], inplace=True)
df1.index=range(len(df1))
df1

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,name1,name2,weight
0,比特幣,期貨,109995.435809
1,比特幣,價格,82999.328568
2,狗狗幣,殺手,54148.417309
3,狗狗幣,教父,47857.626521
4,持有,比特幣,44810.823553
5,狗狗幣,起死回生,29960.267891
6,接受,比特幣,28603.233884
7,表示,比特幣,28372.142365
8,斯克,狗狗幣,27152.592286
9,美元,比特幣,26175.701717


## networkx
* 會有版本版本衝突問題，建議用 nodexl

# 第二回合的配對

In [41]:
new=set(list(df1.name1)+list(df1.name2))
new.remove(kw[0])
new

{'以太',
 '價格',
 '持有',
 '挖礦',
 '接受',
 '教父',
 '斯克',
 '斯克上',
 '暴漲',
 '期貨',
 '殺手',
 '狗狗幣',
 '節禮物',
 '美元',
 '表示',
 '認為',
 '購入',
 '起死回生',
 '養柴'}

## 挑選合於條件的配對

In [42]:
new=['以太'] # !!!!!!!!!!!!!!!!!!!!!!

In [43]:
second=df1[["name1","name2","weight"]].values.tolist()
for n in new:  ####
    W1=[]
    W2=[]
    weight=[]
    for (w1, w2), x_2 in pair_chi_squares.most_common():
        # 設定條件, 在 new 而且第一回合不存在 ########################################################
        if (len(w1)>1 and len(w2)>1) and ((w1 == n and w2 not in kw) or (w2 == n and w1 not in kw)):     
        ############################################################################################    
            W1.append(w1)
            W2.append(w2)
            weight.append(x_2)        
    edges2=list(zip(W1, W2, weight))
    dfs=pd.DataFrame(edges2,columns=["name1","name2","weight"])
    dfs=dfs.sort_values(by=['weight'],ascending=False)
    # 選取最大的十個
    dfs=dfs.iloc[:10]
    add=dfs.values.tolist()
    second+=add
df2=pd.DataFrame(second,columns=["name1","name2","weight"])

In [44]:
df2

,name1,name2,weight
0,比特幣,期貨,109995.435809
1,比特幣,價格,82999.328568
2,狗狗幣,殺手,54148.417309
3,狗狗幣,教父,47857.626521
4,持有,比特幣,44810.823553
5,狗狗幣,起死回生,29960.267891
6,接受,比特幣,28603.233884
7,表示,比特幣,28372.142365
8,斯克,狗狗幣,27152.592286
9,美元,比特幣,26175.701717


# nodexl
* https://www.smrfoundation.org/nodexl/
* 填表申請安裝
* 社會網絡分析超好用軟體
